# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

def scrape_books(min_rating, max_price):
    base_url = "https://books.toscrape.com/catalogue/page-{}.html"
    books_data = []

    # Loop through pages (the website has 50 pages of books)
    for page_num in range(1, 51):
        url = base_url.format(page_num)
        try:
            response = requests.get(url)
            response.raise_for_status()  # Check if the request was successful
            soup = BeautifulSoup(response.text, 'html.parser')
        except requests.exceptions.RequestException as e:
            print(f"Error fetching page {page_num}: {e}")
            continue  # Skip this page and move to the next one
        
        books = soup.find_all('article', class_='product_pod')

        for book in books:
            try:
                # Extract book details
                title = book.find('h3').find('a')['title']
                price = float(book.find('p', class_='price_color').text[1:].replace("£", ""))
                rating = book.find('p', class_='star-rating')['class'][1]
                availability = book.find('p', class_='instock availability').text.strip()

                if 'In stock' not in availability or price > max_price:
                    continue  # Skip books that aren't in stock or are too expensive
                
                rating_map = {'One': 1, 'Two': 2, 'Three': 3, 'Four': 4, 'Five': 5}
                rating_value = rating_map.get(rating, 0)

                # Get genre (assuming it's the last breadcrumb item)
                genre = soup.find('ul', class_='breadcrumb').find_all('li')[-1].text.strip()

                # Get the UPC of the book (Universal Product Code)
                book_page = book.find('h3').find('a')['href']
                book_url = "https://books.toscrape.com/catalogue" + book_page[8:]
                book_response = requests.get(book_url)
                book_soup = BeautifulSoup(book_response.text, 'html.parser')

                # Safe UPC extraction (Handle cases where UPC might not exist)
                upc = None
                upc_element = book_soup.find('th', text='UPC')
                if upc_element:
                    upc = upc_element.find_next_sibling('td').text.strip()

                # Description (if available)
                description = book_soup.find('meta', {'name': 'description'})
                description = description['content'] if description else 'No description available'

                # Only add books that meet the minimum rating
                if rating_value >= min_rating:
                    books_data.append({
                        'UPC': upc if upc else 'N/A',  # If UPC is not found, use 'N/A'
                        'Title': title,
                        'Price (£)': price,
                        'Rating': rating_value,
                        'Genre': genre,
                        'Availability': availability,
                        'Description': description
                    })
            except Exception as e:
                print(f"Error processing book: {e}")
                continue  # Skip this book if there is an issue

        # Add a delay between requests to avoid overwhelming the server
        time.sleep(1)

    # Convert to pandas DataFrame
    books_df = pd.DataFrame(books_data)
    return books_df

# Example usage
df = scrape_books(min_rating=3, max_price=20.0)
print(df.head())


C:\Users\Gerardo\AppData\Local\Temp\ipykernel_22688\408336456.py:48: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  upc_element = book_soup.find('th', text='UPC')


   UPC                                              Title  Price (£)  Rating  \
0  N/A  The Coming Woman: A Novel Based on the Life of...      17.93       3   
1  N/A                                        Set Me Free      17.46       5   
2  N/A  The Four Agreements: A Practical Guide to Pers...      17.66       5   
3  N/A                                     Sophie's World      15.94       5   
4  N/A            Untitled Collection: Sabbath Poems 2014      14.27       4   

          Genre Availability               Description  
0  All products     In stock  No description available  
1  All products     In stock  No description available  
2  All products     In stock  No description available  
3  All products     In stock  No description available  
4  All products     In stock  No description available  
